## NOMS Paper Synthetic (5.1)
**TODO: Move evaluation code to a script and merge this notebook with NOMS_Paper.**

In [ ]:
using CSV
using DataFrames
using Distributions
using HMMBase
using JSON
using ParsimoniousMonitoring
using ProgressMeter
using PyPlot

In [ ]:
using POMDPs
using POMDPModelTools
using DiscreteValueIteration

In [ ]:
include("thesis.jl");

### Evaluation

In [ ]:
MRE(baseline, candidate) = mean((candidate .- baseline) ./ baseline);

In [ ]:
function synthetic_mdp(β, τmax)
    p1 = p2 = HMM([β 1 - β; 1 - β β], [Constant(0.01), Constant(100)])
    MonitoringMDP([p1, p2], [τmax, τmax], [25, 25], 0.99)
end;

In [ ]:
function eval_policies(mdp, policies)
    results = Dict{String,Float64}()

    # Baseline VI policy
    solver = SparseValueIterationSolver(max_iterations = 5000)

    timing = @timed baseline_policy = solve(solver, mdp)
    baseline = evaluate(mdp, baseline_policy).(states(mdp))
    println("Baseline: $(timing[2])s")

    # Candidate policies
    for (name, policy_fn) in policies
        policy = CachedPolicy(mdp, policy_fn(mdp))
        timing = @timed vf = evaluate(mdp, policy).(states(mdp))
        println("$(name): $(timing[2])s")
        results[name] = MRE(baseline, vf)
    end

    results
end;

### Main

In [ ]:
policies = Dict{String,Function}(
    # "Greedy" => mdp -> GreedyPolicy(mdp),
    "Greedy" => mdp -> AnalyticalGreedyPolicy(mdp),
    "RH-2" => mdp -> RecedingHorizonPolicy(mdp, 2, shared_cache = true),
    "RH-3" => mdp -> RecedingHorizonPolicy(mdp, 3, shared_cache = true),
    "RH-4" => mdp -> RecedingHorizonPolicy(mdp, 4, shared_cache = true),
    "Heuristic" => mdp -> HeuristicPolicy(mdp, SparseValueIterationSolver(max_iterations = 100, include_Q = false))
);

In [ ]:
# βs = vcat(0.01:0.01:0.05, 0.1:0.1:0.9, 0.95:0.01:0.99)
# results = Vector{Dict}(undef, size(βs))

# for i in eachindex(βs)
#     mdp = synthetic_mdp(βs[i], 150)
#     results[i] = eval_policies(mdp, policies)
# end

In [ ]:
# df = DataFrame([Float64[], String[], Float64[]], [:β, :policy, :mre])
# for (β, result) in zip(βs, results)
#     for (policy, mre) in result
#         push!(df, (β, policy, mre))
#     end
# end
# df = unstack(df, :policy, :mre)
# CSV.write("../results/noms_synthetic.csv", df);

In [ ]:
df = CSV.read("../results/noms_synthetic.csv")
first(df, 6)

In [ ]:
fig, ax = subplots(figsize = (7,5))
for (policy, label) in Dict("Greedy" => "Glouton", "Heuristic" => "Heuristique", "RH-2" => "RH-2", "RH-3" => "RH-3", "RH-4" => "RH-4")
    ax.plot(df[:,:β], df[:,Symbol(policy)] * 100, label = label)
end
ax.grid()
ax.legend(loc = "upper left", ncol = 5)
ax.set(xticks = 0:0.1:1, yticks = 0:10:70, xlabel = L"$\beta$", ylabel = "MRE (%)")
save_thesis("heuristics_mre", clean = true, hwr = 0.7)